<h1>Bussiness undderstanding & problem.</h1>

Di Negara Amerika Serikat di Kota New York, semua kendaraan taksi di kelola oleh TLC(Taxi and Limousine Commission). Didirikan sejak tahun 1971. New York TLC adalah agensi yang bertanggung jawab untuk melisensikan dan mengatur taksi Medallion di Kota New York (Berwana Kuning), kendaraan untuk disewa secara khusus (khusus). Salah satu vendor dari TLC ini adalah VeriFone Inc.

Dalam industri taksi, memahami pola permintaan pelanggan, preferensi wilayah adalah kunci untuk mengoptimalkan operasi dan meningkatkan profitabilitas. Dengan menilai kapan dan di mana layanan paling dibutuhkan serta apa yang mendorong pelanggan memberikan apresiasi lebih, perusahaan dapat membuat keputusan yang lebih tepat tentang alokasi sumber daya, penetapan harga, dan inisiatif pelayanan pelanggan. Meningkatkan profitabilitas layanan taksi dengan menganalisis perilaku dan preferensi pelanggan merupakan sebuah tantangan kompleks di era kompetisi digital yang ketat saat ini. Pemahaman mendalam tentang dinamika operasional dan pelayanan terhadap pelanggan menjadi fondasi utama untuk mengatasi tantangan ini. Dengan mengeksplorasi lebih dalam aspek-aspek tertentu dari operasional taksi, kita dapat menemukan solusi yang lebih spesifik dan terarah. Salah satu dari aspek tersebut adalah:

- Variabilitas permintaan berdasarkan beberapa aspek seperti waktu, wilayah, dan prefensi pelanggan.

<h1>Goals</h1>

Memberikan insight terhadap stakeholder (Manajer dari VeriFone Inc.) untuk membantu dalam strategi atau pembuatan keputusan terkait hal-hal berikut :

- Meningkatkan profit melalui penjadwalan armada taksi.
- bagaimana cara memindahkan metode pembayaran penumpang dari pembayaran cash dan lainnya menjadi credit, dan bagaimana perusahaan bisa mendapatkan keuntungan dari perpindahan metode pambayaran tersebut.
- Perusahaan ingin para penumpang merubah cara penumpang memanggil taksi dari yang sebelumny langsung di jalan menjadi panggilan lewat sarana lain seperti apps.


<h1>New York City TLC Trip Record Data Dictionary</h1>

Klik link https://drive.google.com/drive/folders/1NYHIL-RgVPW-HONz4pdzlcbIChF-c37N

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import display
alt.data_transformers.enable('default', max_rows=None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Mengimport dataset

df = pd.read_csv('NYC TLC Trip Record.csv')

# Melihat isi 5 teratas , terbawah, dan bentuk dari dataset

display(df.head(),df.tail())
df.shape

Selanjutnya sebelum <i>skimming data</i> atau melihat data secara sekilas. Data kolom <b>VendorID</b> akan difilter sesuai yang dibutuhkan oleh <i>stackholder</i>, dengan memilih <b>VendorID</b> = 2. Setelah itu kolom <b>VendorID</b> bisa di hapus karena sudah tidak digunakan lagi.

In [ ]:
# Memfilter kolom VendorID
df = df[(df['VendorID'] == 2)]

# Menghapus kolom VendorId
df = df.reset_index().drop(['VendorID', 'index'], axis=1)

df

Setelah itu, menghapus tiga kolom yang tidak ada di dalam dictionary dan beberapa kolom yang tidak diperlukan dalam dataset New York City TLC Trip Record karena tidak relevan dengan pertanyaan atau permintaan dari stockholder.

In [ ]:
# Menghapus yang tidak ada dalam dictionary.
df = df.drop(['extra', 'ehail_fee', 'congestion_surcharge'], axis=1)

# Menghapus kolom yang tidak diperlukan.
df = df.drop(['store_and_fwd_flag', 'fare_amount', 'mta_tax', 'tolls_amount', 'improvement_surcharge', ], axis=1)

# Menampilkan 5 data teratas dan terbawah serta shape baru dari data.
display(df.head(), df.tail())
df.shape

<h1>Skimming Data</h1>

Melihat sekilas dataset NYC TLC Trip Record.

In [ ]:
# Melihat data secara detail.
pd.set_option('display.max_colwidth', None)

pd.DataFrame({
    'feature': df.columns.values,
    'data_type': df.dtypes.values,
    'null_value(%)': df.isna().mean().values * 100,
    'neg_value(%)': [len(df[col][df[col] < 0]) / len(df) * 100 if col in df.select_dtypes(include=[np.number]).columns else 0 for col in df.columns],
    '0_value(%)': [len(df[col][df[col] == 0]) / len(df) * 100 if col in df.select_dtypes(include=[np.number]).columns else 0 for col in df.columns],
    'duplicate' : df.duplicated().sum(),
    'n_unique': df.nunique().values,
    'sample_unique': [df[col].unique() for col in df.columns]}
).round(3)

In [ ]:
# Menghitung nullity correlation matrix.
nullity_corr = df.isnull().corr()

# Melt correlation matrix nya.
nullity_corr_melted = nullity_corr.reset_index().melt(id_vars='index', var_name='column', value_name='correlation')

# Membuat heatmap menggunakan Altair.
heatmap = alt.Chart(nullity_corr_melted).mark_rect().encode(
    x=alt.X('index:O', title=''),
    y=alt.Y('column:O', title=''),
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme='viridis'), title='Nullity Correlation'),
    tooltip=['index', 'column', 'correlation']
).properties(
    width=500,
    height=500,
    title='Nullity Correlation Heatmap'
)

heatmap

Berdasarkan <i>heatmap</i> di atas, relasi <i>missing value</i> yang bernilai satu yang artinya ketika terdapat kolom <i>missing value</i> pada baris tertentu maka kolom lainnya juga <i>missing</i>

<h1>Exploratory Data Analysis.</h1>

In [ ]:
# Distribusi data.

df.describe().T

Berdasarkan data di atas dapat di simpulkan bahwa terdapat missing values yang bisa kita lihat dari count, min values bisa dilihat dari min, outliers bisa dilihat dari max di beberapa kolom.

In [ ]:
# Mmebuat list untuk menyimpan chart.
charts = []

# loop setiap numerical kolom dan membuat box plot.
for col in df.select_dtypes(include=[int, float]):
    chart = alt.Chart(df).mark_boxplot().encode(
        y=alt.Y(col, title=col)
    ).properties(
        width=500,
        height=500
    )
    charts.append(chart)

# Concat chart nya secara horizontal.
alt.hconcat(*charts)

In [ ]:
# Menghitung Spearman correlation matrix
corr_matrix = df.corr('spearman')

# Mereset index dan melt correlation matrix nya.
corr_melted = corr_matrix.reset_index().melt(id_vars='index', var_name='column', value_name='correlation')

# Membuat heatmap menggunkan Altair
heatmap = alt.Chart(corr_melted).mark_rect().encode(
    x=alt.X('index:O', title=''),
    y=alt.Y('column:O', title=''),
    color=alt.Color('correlation:Q', scale=alt.Scale(domain=[-1, 0, 1], range=['blue', 'white', 'red']), title='Spearman Correlation'),
    tooltip=['index', 'column', 'correlation']
).properties(
    width=700,
    height=700,
    title='Spearman Correlation Heatmap'
)

heatmap

Dari data di atas kita dapat melihat korelasi2 dari data atau kolom yang ada.

<h1>Data Cleaning</h1>

<h3>Duplicate Value</h3>

Mencari dan menangani data yang ada duplikat nya.

In [ ]:
df[df.duplicated()]

In [ ]:
print(df[df.duplicated()].sum())
print(df.duplicated().value_counts())

Data di atas menunjukan tidak ada data yang duplikat.

In [ ]:
duplicate_value = df[df.duplicated(subset=['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance'], keep=False)]
duplicate_value

In [ ]:
dup_negatif_val = duplicate_value[df['total_amount']<0]
dup_negatif_val 

In [ ]:
df.drop(dup_negatif_val.index, inplace=True)

In [ ]:
df[df.duplicated(subset=['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance'])]

Di atas saya mencoba mencari secara lebih spesifik data duplikat nya, dengan cara mengambil beberapa kolom saja. dan hasil nya ternyata ada dan langsung saya tangani. 

<h3>Negative Values</h3>

Mencari data yang bernilai negatif.

In [ ]:
pd.DataFrame({
    'feature': df.columns.values,
    'neg_value(%)': [len(df[col][df[col] < 0]) / len(df) * 100 if np.issubdtype(df[col].dtype, np.number) else 0 for col in df.columns]
}).round(3)

Masih ada nilai negatif di kolom <b>total_amount</b>. Nilai dari kolom tersebut harus diubah menjadi positif atau di absolutkan.

In [ ]:

negatif_val  = df[df['total_amount']<0]
df.loc[negatif_val.index, 'total_amount'] = df['total_amount'].abs()

Saya memfilter data yang bernilai negatif dan saya absolutkan nilai nya.

In [ ]:
# Melihat apakah nilai negatif nya sudah hilang.

pd.DataFrame({
    'feature': df.columns.values,
    'neg_value(%)': [len(df[col][df[col] < 0]) / len(df) * 100 if np.issubdtype(df[col].dtype, np.number) else 0 for col in df.columns]
}).round(3)

<h3>Zero Values</h3>

Mencari nilai kosong/nol.

In [ ]:
pd.DataFrame({
    'feature': df.columns.values,
    'data_type': df.dtypes.values,
    '0_value(%)': [len(df[col][df[col] == 0]) / len(df) * 100 if np.issubdtype(df[col].dtype, np.number) else 0 for col in df.columns]
}).round(3)

Ada nilai kosong di kolom <b>passenger_count</b>trip_distance<b>tip_amount</b><b>total_amount</b>.

In [ ]:
df['passenger_count'] = df['passenger_count'].replace(0, df['passenger_count'].median())
df = df[df['total_amount']>0]
df = df[df['trip_distance']>0]

In [ ]:
pd.DataFrame({
    'feature': df.columns.values,
    'data_type': df.dtypes.values,
    '0_value(%)': [len(df[col][df[col] == 0]) / len(df) * 100 if np.issubdtype(df[col].dtype, np.number) else 0 for col in df.columns]
}).round(3)

<h1>Missing Values</h1>

Mencari missing values atau Null.

In [ ]:
pd.DataFrame({
    'feature': df.columns.values,
    'data_type': df.dtypes.values,
    'null_value(sum)': df.isna().sum(),
    'null_value(%)': df.isna().mean().values * 100
}).round(3)

Ada <i>missing values</i> atau Null di kolom <b>RatecodeID</b>, <b>passenger_count</b> <b>payment_type</b>, <b>trip_type</b>.

Mengisi <i>missing values</i> pada kolom <b>RatecodeID</b>

In [ ]:
df['RatecodeID'].sum()

In [ ]:
df['RatecodeID'].value_counts()

In [ ]:
df['RatecodeID'].isna().sum()

In [ ]:
# Untuk baris dimana PULoacationID  dan DOLocationID ada nilainya.
for index, row in df[(~df['PULocationID'].isnull()) & (~df['DOLocationID'].isnull()) & (df['RatecodeID'].isnull())].iterrows():
    # Cari baris yang PULocationID dan DOLocationID dan non-null RatecodeID.
    similar_rows = df[(df['PULocationID'] == row['PULocationID']) & (df['DOLocationID'] == row['DOLocationID']) & (~df['RatecodeID'].isnull())]
    if not similar_rows.empty:
        # Fill missing RatecodeID with the most common RatecodeID for similar trips
        most_common_ratecode = similar_rows['RatecodeID'].mode()[0]
        df.at[index, 'RatecodeID'] = most_common_ratecode

# Untuk baris yang hanya ada salah satu dari PULocationID atau DOLocationID tersedia nilainya.
for index, row in df[((~df['PULocationID'].isnull()) | (~df['DOLocationID'].isnull())) & (df['RatecodeID'].isnull())].iterrows():
    # Cari baris yang sama nilainya dengan PULocationID atau DOLocationID dan dengan non-null RatecodeID.
    if not pd.isnull(row['PULocationID']):
        similar_rows = df[(df['PULocationID'] == row['PULocationID']) & (~df['RatecodeID'].isnull())]
    else:
        similar_rows = df[(df['DOLocationID'] == row['DOLocationID']) & (~df['RatecodeID'].isnull())]
    if not similar_rows.empty:
        # Isi missing values RatecodeID dengan modus RatecodeID untuk trips yang sama.
        most_common_ratecode = similar_rows['RatecodeID'].mode()[0]
        df.at[index, 'RatecodeID'] = most_common_ratecode

In [ ]:
# Menghapus missing values RatecodeID yang tersisa.
df.dropna(subset=['RatecodeID'], inplace=True)

In [ ]:
df['RatecodeID'].sum()

In [ ]:
df['RatecodeID'].isna().sum()

In [ ]:
df['RatecodeID'].value_counts()

In [ ]:
pd.DataFrame({
    'feature': df.columns.values,
    'data_type': df.dtypes.values,
    'null_value(sum)': df.isna().sum(),
    'null_value(%)': df.isna().mean().values * 100
}).round(3)

Mengisi <i>missing values</i> pada kolom <b>passenger_count</b>

In [ ]:
df['passenger_count'].fillna(df['passenger_count'].median(), inplace=True)

print(df['passenger_count'].value_counts())
print('Total NaN values: ', len(df[df['passenger_count'].isna()]))

Mengsisi <i>missing values</i> di kolom passenger_count dengan median nya.

In [ ]:
pd.DataFrame({
    'feature': df.columns.values,
    'data_type': df.dtypes.values,
    'null_value(sum)': df.isna().sum(),
    'null_value(%)': df.isna().mean().values * 100
}).round(3)

Mengisi <i>missing values</i> pada kolom <b>payment_type</b>

In [ ]:
print(df['payment_type'].value_counts())
print('Total NaN values: ', len(df[df['payment_type'].isna()]))

In [ ]:
df[(df['tip_amount']>0)]['payment_type'].value_counts()

In [ ]:
df.loc[df['payment_type'].isna(), 'payment_type'] = df['tip_amount'].apply(lambda x: 1.0 if x > 0.0 else 5.0)
df['payment_type'].value_counts()

Dari <i>dictionary</i> kolom saya mengetahui kalau penumpang memberikan tip maka dia pasti menggunakan kredit sebagai <b>payment_type</b> nya, jadi di sini saya menggunakan <b>tip_amount</b> sebagai filter untuk mengisi <b>payment_type</b> yang kosong.

Mengisi <i>missing values</i> pada kolom <b>trip_type</b>

In [ ]:
df.groupby(['RatecodeID', 'trip_type']).size().reset_index(name='Count')

In [ ]:
df[df['trip_type'].isna()].groupby('RatecodeID').size().reset_index(name='count')

In [ ]:
df['trip_type'].fillna(1, inplace=True)
df['trip_type'].value_counts()

Saya mengis <i>missing values</i> di kolom <b>trip_type</b> dengan menggunakan kolom <b>RatecodeID</b>, dimana jika <b>RatecodeID</b> nya 1, 2, dan 3 maka <b>trip_type</b> nya ada 1 selain itu <b>trip_type</b> nya adalah 2.

<h1>Formatting and Outliers Handling</h1>

Mengformat dan menangani anomali di dataset

<h3>lpep_pickup_datetime and lpep_dropoff_datetime</h3>

In [ ]:
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

Mengubah tipe data <i>object</i> dari kolom <b>lpep_pickup_datetime</b> dan <b>lpep_dropoff_datetime</b> menjadi <i>datetime</i>

In [ ]:
outliers = df[(df['lpep_pickup_datetime'].dt.month!=1) | (df['lpep_pickup_datetime'].dt.year!=2023)]
outliers

In [ ]:
df.drop(outliers.index, inplace=True)

In [ ]:
df[(df['lpep_pickup_datetime'].dt.month!=1) | (df['lpep_pickup_datetime'].dt.year!=2023)]

Menemukan anomali di mana tahun dan bulan di kolom <b>lpep_pickup_datetime</b> dan <b>lpep_dropoff_datetime</b> bukan tahun 2023 dan bulan januari.

In [ ]:

time_bins = [0, 5, 11, 15, 20, 24] 
time_labels = ['Midnight', 'Morning', 'Noon', 'Evening', 'Night']
df['pickup_time'] = pd.cut(df['lpep_pickup_datetime'].dt.hour, bins=time_bins, labels=time_labels, right=False)

df['day_category'] = df['lpep_pickup_datetime'].dt.day_name().apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekdays')

Membuat kategori untuk waktu dan hari.

<h3>RatecodeID</h3>

In [ ]:
df['RatecodeID'] = df['RatecodeID'].replace({1:'Standard rate', 2:'JFK Airport', 3:'Newark Airport', 4:'Nassau or Westchester', 5:'Negotiated fare', 6:'Group ride'})
df['RatecodeID'].value_counts()

Mengubah isi dari kolom <b>RatecodeID</b>.

<h3>passenger_count	</h3>

In [ ]:
df['passenger_count'].value_counts()

In [ ]:

alt.Chart(df).mark_boxplot().encode(
    y=alt.Y('passenger_count:O', title='Total Passenger'),
    x=alt.X('passenger_count:Q', title='Count'),
).properties(
    width=1000,
    height=500,
    title='Passenger Distribution'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

In [ ]:
df['passenger_count'] = df['passenger_count'].apply(lambda x: '>3' if x>3 else str(x))
df['passenger_count'].value_counts()

Membuat kategori untuk <b>passenger_count</b>.

<h3>trip_distance</h3>

In [ ]:

alt.Chart(df).mark_boxplot().encode(
    x=alt.X('trip_distance:Q', title='Distance (mile)'),
).properties(
    width=1000,
    height=500,
    title='Trip Distance Distribution'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

In [ ]:
# Calculate quartiles and IQR
q1 = df['trip_distance'].quantile(0.25)
q3 = df['trip_distance'].quantile(0.75)
iqr = q3 - q1

# Define outlier bounds
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Find outliers
outliers = df[(df['trip_distance'] < lower_bound) | (df['trip_distance'] > upper_bound)]

#  Median trip_distance. 
median = df['trip_distance'].median()
outliers

Mencari anomali data menggunakan IQR, dan mencari median dari kolom <b>trip_distance</b>.

In [ ]:
df['trip_distance'] = df['trip_distance'].where((df['trip_distance'] >= lower_bound) & (df['trip_distance'] <= upper_bound), median)

df.head()

Mengganti anomali data dengan median kolom <b>trip_distance</b>.

In [ ]:
bins_of_distance = [0, 2, 4, 6, np.inf] 
labels_of_distance = ['<2 miles', '3-4 miles', '5-6 miles', '>6 miles']

df['distance_bins'] = pd.cut(df['trip_distance'], bins=bins_of_distance, labels=labels_of_distance)

In [ ]:
df['distance_bins'].value_counts()

Mengkategorikan kolom <b>trip_distance</b>.

<h3>Trip_Duration</h3>

Membuat kolom baru yang isi kolom nya adalah berapa lama perjalanan taxi. 

In [ ]:
diff = (df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']).dt.total_seconds()/60
df['trip_duration'] = diff.round(2)

In [ ]:
# Create an Altair chart
alt.Chart(df).mark_boxplot().encode(
    x=alt.X('trip_duration:Q', title='Duration (minutes)'),
).properties(
    width=1000,
    height=500,
    title='Trip Duration Distribution'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

In [ ]:
# Calculate quartiles and IQR
q1 = df['trip_duration'].quantile(0.25)
q3 = df['trip_duration'].quantile(0.75)
iqr = q3 - q1

# Define outlier bounds
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Find outliers
outliers = df[(df['trip_duration'] < lower_bound) | (df['trip_duration'] > upper_bound)]

#  Median trip_distance. 
median = df['trip_duration'].median()
outliers

In [ ]:
df['trip_duration'] = df['trip_duration'].where((df['trip_duration'] >= lower_bound) & (df['trip_duration'] <= upper_bound), median)

df.head()

In [ ]:
bins_of_duration = [0, 10, 20, 30, np.inf] 
labels_of_duration = ['<10 minutes', '11-20 minutes', '21-30 minutes', '>30 minutes']

df['duration_bins'] = pd.cut(df['trip_duration'], bins=bins_of_duration, labels=labels_of_duration)

In [ ]:
df['duration_bins'].value_counts()

<h3>total_amount</h3>

In [ ]:
df.head()

In [ ]:
# Calculate quartiles and IQR
q1 = df['total_amount'].quantile(0.25)
q3 = df['total_amount'].quantile(0.75)
iqr = q3 - q1

# Define outlier bounds
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Find outliers
outliers = df[(df['total_amount'] < lower_bound) | (df['total_amount'] > upper_bound)]

#  Median trip_distance. 
median = df['total_amount'].median()
outliers

In [ ]:
df['total_amount'] = df['total_amount'].where((df['total_amount'] >= lower_bound) & (df['total_amount'] <= upper_bound), median)

df.head()

<h3>tip_amount</h3>

In [ ]:
# Calculate quartiles and IQR
q1 = df['tip_amount'].quantile(0.25)
q3 = df['tip_amount'].quantile(0.75)
iqr = q3 - q1

# Define outlier bounds
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Find outliers
outliers = df[(df['tip_amount'] < lower_bound) | (df['tip_amount'] > upper_bound)]

#  Median trip_distance. 
median = df['tip_amount'].median()
outliers

In [ ]:
df['tip_amount'] = df['tip_amount'].where((df['tip_amount'] >= lower_bound) & (df['tip_amount'] <= upper_bound), median)

df.head()

In [ ]:
df['tip_amount'].max()

In [ ]:
bins_tip = [-np.inf, 0, 2, 4, 6, np.inf]
labels_tip = ['0$', '1-2$', '3-4$', '5-6$', '>8$']
df['tip_amount_bins'] = pd.cut(df['tip_amount'], bins=bins_tip, labels=labels_tip)

In [ ]:

df['tip_amount_bins'].value_counts()

<h3>payment_type</h3>

In [ ]:
df['payment_type']=df['payment_type'].replace({1:'Credit card', 2:'Cash', 3:'No charge', 4:'Dispute', 5:'Unknown', 6:'Voided trip'})
df['payment_type'].value_counts()

<h3>trip_type<h3>

In [ ]:
df['trip_type']=df['trip_type'].replace({1:'Street-hail', 2:'Dispatch'})
df['trip_type'].value_counts()

<h1>Setting Columns Index</h1>

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df = df[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'day_category', 'pickup_time', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'distance_bins','total_amount', 'tip_amount', 'tip_amount_bins', 'payment_type', 'trip_type', 'trip_duration', 'duration_bins']]

df

In [ ]:
df.to_csv('cleaned_data.csv', index=False)

<h1>Data Analys<h1>

<h1>Skimming</h1>

In [ ]:
pd.DataFrame({
    'feature': df.columns.values,
    'data_type': df.dtypes.values,
    'null_value(%)': df.isna().mean().values * 100,
    'neg_value(%)': [len(df[col][df[col] < 0]) / len(df) * 100 if col in df.select_dtypes(include=[np.number]).columns else 0 for col in df.columns],
    '0_value(%)': [len(df[col][df[col] == 0]) / len(df) * 100 if col in df.select_dtypes(include=[np.number]).columns else 0 for col in df.columns],
    'duplicate' : df.duplicated().sum(),
    'n_unique': df.nunique().values,
    'sample_unique': [df[col].unique() for col in df.columns]}
).round(3)

<h1>Exploratory Data Analysis</h1>

<h3>Categorical</h3>

In [ ]:
cols = ['day_category', 'pickup_time', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 
        'distance_bins', 'tip_amount_bins', 'payment_type', 'trip_type', 
        'duration_bins']

# Menentukan warna.
color = "steelblue"

# Membuat chart dengan altair.
charts = []
for col in cols:
    chart = alt.Chart(df).mark_bar().encode(
        x=alt.X(col, title=col),
        y=alt.Y('count()', title='Count'),
        color=alt.value(color)
    ).properties(
        width=700,
        height=500,
        title=col
    )
    
    # Menambahkan text di atas setiap bar.
    text = chart.mark_text(
        align='center',
        baseline='bottom',
    ).encode(
        text='count()'
    )
    
    charts.append(chart + text)

# Menggabungkan setiap chart menjadi grid.
grid = alt.vconcat(*[alt.hconcat(*charts[i:i+3]) for i in range(0, len(charts), 3)])

#  Menampilkan grid nya.
grid

<h5>Insight</h5>

- Data tidak ada yang terdistribusi normal.

In [ ]:
trip_monthly = df.groupby(df['lpep_pickup_datetime'].dt.date).agg(trip_bydate=('lpep_pickup_datetime', 'count')).reset_index()
trip_monthly['lpep_pickup_datetime'] = pd.to_datetime(trip_monthly['lpep_pickup_datetime'])
trip_monthly['day_of_week'] = trip_monthly['lpep_pickup_datetime'].dt.day_name()

trip_dayly = trip_monthly.groupby('day_of_week').agg(trip_byday=('trip_bydate', 'sum'), day_count=('day_of_week', 'count'), avg_trip=('trip_bydate', 'mean')).reset_index()
trip_dayly['day_cat']= trip_dayly['day_of_week'].apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekdays')

trip_dayly_cat = trip_dayly.groupby('day_cat').agg(trip_bycat=('trip_byday', 'sum')).reset_index()

trip_merge = pd.merge(trip_monthly, trip_dayly, on='day_of_week', how='left')
trip_merge = pd.merge(trip_merge, trip_dayly_cat, on='day_cat', how='left')

In [ ]:
# Memilih warna garis.
color = ["#1f77b4", "#ff7f0e"]

# Membentuk ulang data untuk di plot
trip_merge_melted = trip_merge.melt(id_vars='lpep_pickup_datetime', value_vars=['trip_bydate', 'avg_trip'])

# Membuat Altair chart.
line_chart = alt.Chart(trip_merge_melted).mark_line().encode(
    x=alt.X('lpep_pickup_datetime:T', title='Date'),
    y=alt.Y('value:Q', title='Number of trips'),
    color=alt.Color('variable:N', title='Trip Type', scale=alt.Scale(range=color), legend=alt.Legend(title=None))
).properties(
    width=2000,
    height=500,
    title='Trip Distribution in January'
)

# Menampilkan chart.
line_chart.interactive()

<h5>Insight</h5>

- Jumlah perjalanan taxi terendah berada di tanggal 1 Januari hari minggu.
- Jumlah perjalanan taxi tertinggi berada di tanggal 25 Januari hari rabu.
- Terdapat pola yang berulang.

In [ ]:
# Meengkonversi format data agar bisa di pakai altair.
alt_data = pd.DataFrame({
    'day_of_week': trip_merge['day_of_week'].unique().tolist(),
    'avg_trip': trip_merge['avg_trip'].unique().tolist(),
    'trip_byday': trip_merge['trip_byday'].unique().tolist()
})

# Membuat bar chart untuk AVG trips.
bar_avg_trip = alt.Chart(alt_data).mark_bar().encode(
    x=alt.X('day_of_week:N', title='Day of Week'),
    y=alt.Y('avg_trip:Q', title='Average Trips'),
    color=alt.Color('day_of_week:N', scale=alt.Scale(range=color), title='Day of Week'),
    tooltip=['day_of_week:N', 'avg_trip:Q']
).properties(
    title='Average Trips',
    width=920,
    height=500
)

# Membuat text label.
text_avg_trip = bar_avg_trip.mark_text(
    align='center',
    baseline='bottom',
    dy=-5,
    color='black',
    fontSize=10
).encode(
    text=alt.Text('avg_trip:Q', format='.0f')
)

# Membuat bar chart untuk total trips.
bar_total_trip = alt.Chart(alt_data).mark_bar().encode(
    x=alt.X('day_of_week:N', title='Day of Week'),
    y=alt.Y('trip_byday:Q', title='Proportion of Total Trips'),
    color=alt.Color('day_of_week:N', scale=alt.Scale(range=color), title='Day of Week'),
    tooltip=['day_of_week:N', 'trip_byday:Q']
).properties(
    title='Proportion of Total Trips',
    width=920,
    height=500
)

# Membuat text label.
text_total_trip = bar_total_trip.mark_text(
    align='center',
    baseline='bottom',
    dy=-5,
    color='black',
    fontSize=10
).encode(
    text=alt.Text('trip_byday:Q', format='.0f')
)

# Menggabungkan bar chart dengan text label.
(bar_avg_trip + text_avg_trip) | (bar_total_trip + text_total_trip)

<h5>Insight</h5>

- Proporsi tertinggi total trip selama bulan januari yaitu hari Selasa sebesar 9535.
- Meski demikian, berdasarkan rata-rata banyaknya perjalanan perhari, perjalanan perhari tertinggi terjadi pada hari kamis mencapai 2123 perjalanan perhari.
- Ketika weekday jumllah perjalanan naik, ketika memasuki weekend jumlah perjalanan menurun.

In [ ]:
# Menentukan urutan dari hari.
day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Buat tab silang untuk mendapatkan total perjalanan per hari dan jam.
cross = pd.crosstab(index=df['lpep_pickup_datetime'].dt.day_name(), 
                    columns=df['lpep_pickup_datetime'].dt.hour, 
                    margins=True).reindex(day_order)

# Reset index dam melt the DataFrame untuk plotting.
melted_cross = cross.iloc[:,:-1].reset_index().melt(id_vars='lpep_pickup_datetime', 
                                                    var_name='hour', 
                                                    value_name='trips')

# mengubah nama kolom
melted_cross.columns = ['Day of Week', 'Hour of the Day', 'Trips']

In [ ]:
# Line plot untuk total trips per hour.
line_plot = alt.Chart(melted_cross).mark_line().encode(
    x='Hour of the Day:O',
    y='Trips:Q',
    color='Day of Week:N',
    tooltip=['Hour of the Day:O', 'Trips:Q']
).properties(
    width=2000,
    height=500,
    title='Total Trips per Hour'
).interactive()

# Heatmap untuk total trips by day and hour.
heatmap = alt.Chart(melted_cross).mark_rect().encode(
    x=alt.X('Hour of the Day:O', title='Hour of the Day'),
    y=alt.Y('Day of Week:O', title='Day of the Week', sort=day_order),
    color=alt.Color('Trips:Q', scale=alt.Scale(scheme='blueorange'), title='Total Trips'),
    tooltip=['Hour of the Day:O', 'Day of Week:O', 'Trips:Q']
).properties(
    width=2000,
    height=500,
    title='Total Trips by Day and Hour'
)

# Menambah text mark untuk akngka trips di setiap kotak.
text = heatmap.mark_text(baseline='middle').encode(
    text='Trips:Q',
    color=alt.condition(
        alt.datum['Trips'] > cross.max().max() / 2,
        alt.value('white'),
        alt.value('black')
    )
)

# Concatenate kedua charts secara vertikal.
alt.vconcat(line_plot, (heatmap + text))

<h5>Insight</h5>

- Jumlah perjalanan terendah terjadi pada jam 04:00 pagi di hari Rabu dengan jumlah perjalanan sebanyak 18 kali.
- Jumlah perjalanan tertinggi terjadi pada jam 18:00 sore di hari Kamis dengan jumlah perjalanan sebanyak 754 kali.
- pada weekday atau hari senin sampai jumat dari siang sampai sore mengalami peningkatan perjalanan taxi.
- Pada weekday atau hari senin sampai jumat jam-jam yang banyak atau ramai perjalanan taksi nya itu berada di jam 15:00 sampai jam 18:00.

In [ ]:
# Group by PULocationID dan mengagregat count dari trips.
agg_zone = df.groupby('PULocationID').size().reset_index(name='count')

# Sort agregat dari DataFrame dengan count dalam descending order.
agg_zone_sorted = agg_zone.sort_values(by='count', ascending=False)

# Pilih 10 teratas pickup zone.
agg_zone_top = agg_zone_sorted.head(10)

# Pilih 10 terbawah pickup zones.
agg_zone_bottom = agg_zone_sorted.tail(10)

# Membuat bar chart untuk 10 teratas pickup zone. 
bars_top = alt.Chart(agg_zone_top).mark_bar().encode(
    y=alt.Y('PULocationID:N', title='Pick Up Zone'),
    x=alt.X('count:Q', title='Total Trips'),
    color=alt.Color('PULocationID:N', scale=alt.Scale(scheme='category20')),
    tooltip=['PULocationID:N', 'count:Q']
)

# Menambahkan text label untuk 10 teratas pickup zone.
text_top = bars_top.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Menggeser text ke kanan agar tidak overlap dengan bar.
).encode(
    text='count:Q'
)

# Gabungkan diagram batang dan label teks untuk 10 Zona Pengambilan teratas.
chart_top = (bars_top + text_top).properties(
    width=2000,
    height=500,
    title='Top 10 Trip Distribution based on Pick Up Zone'
)

# Buat diagram batang untuk 10 Zona Pengambilan.
bars_bottom = alt.Chart(agg_zone_bottom).mark_bar().encode(
    y=alt.Y('PULocationID:N', title='Pick Up Zone'),
    x=alt.X('count:Q', title='Total Trips'),
    color=alt.Color('PULocationID:N', scale=alt.Scale(scheme='category20')),
    tooltip=['PULocationID:N', 'count:Q']
)

# Tambahkan label teks ke bilah untuk 10 Zona Pengambilan terbawah
text_bottom = bars_bottom.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Menggeser text ke kanan agar tidak overlap dengan bar.
).encode(
    text='count:Q'
)

# Gabungkan diagram batang dan label teks untuk 10 Zona Pengambilan terbawah.
chart_bottom = (bars_bottom + text_bottom).properties(
    width=2000,
    height=500,
    title='Bottom 10 Trip Distribution based on Pick Up Zone'
)

# Concatenate ke dua chart secara vertikal.
alt.vconcat(chart_top, chart_bottom)

<h5>Insight</h5>

- Pick up zone dengan trip terbanyak ada di pick up zone id 74 dengan jumlah trip sebanyak 10.626 kali.
- Pick up zone dengan trip terendah ada di pick up zone id 23, 27, 31, 67, 114, 154, 161, 194, 221, dan 245 dengan jumlah trip sebanyak 1 kali.

In [ ]:
# Mengagregat data untuk RatecodeID.
agg_ratecode = df.groupby('RatecodeID').size().reset_index(name='count')
agg_ratecode['RatecodeID'] = agg_ratecode['RatecodeID'].apply(lambda x: x if x == 'Standard rate' else 'Other')
agg_ratecode = agg_ratecode.groupby('RatecodeID').agg(count=('count', 'sum')).sort_values(by='count', ascending=False).reset_index()

# Mengagregat data untuk payment type.
agg_payment = df.groupby('payment_type').size().reset_index(name='count')
agg_payment['payment_type'] = agg_payment['payment_type'].apply(lambda x: x if x in ['Credit card', 'Cash'] else 'Other')
agg_payment = agg_payment.groupby('payment_type').agg(count=('count', 'sum')).sort_values(by='count', ascending=False).reset_index()

# Mengagregat data untuk trip type.
agg_triptype = df.groupby('trip_type').size().reset_index(name='count')

# Buat diagram batang untuk proporsi RatecodeID.
bar_ratecode = alt.Chart(agg_ratecode).mark_bar().encode(
    x=alt.X('RatecodeID:N', title='RatecodeID'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.Color('RatecodeID:N', scale=alt.Scale(range=color)),
    tooltip=['RatecodeID:N', 'count:Q']
).properties(
    width=580,
    height=500,
    title='RatecodeID Proportion'
)

# Buat diagram batang untuk proporsi jenis pembayaran.
bar_payment = alt.Chart(agg_payment).mark_bar().encode(
    x=alt.X('payment_type:N', title='Payment Type'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.Color('payment_type:N', scale=alt.Scale(range=color)),
    tooltip=['payment_type:N', 'count:Q']
).properties(
    width=580,
    height=500,
    title='Payment Type Proportion'
)

# Buat diagram batang untuk proporsi jenis perjalanan.
bar_triptype = alt.Chart(agg_triptype).mark_bar().encode(
    x=alt.X('trip_type:N', title='Trip Type'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.Color('trip_type:N', scale=alt.Scale(range=color)),
    tooltip=['trip_type:N', 'count:Q']
).properties(
    width=580,
    height=500,
    title='Trip Type Proportion'
)

# Gabungkan diagram batang secara horizontal dan tambahkan hitungan di atas setiap batang.
alt.hconcat(
    bar_ratecode + bar_ratecode.mark_text(align='center', baseline='bottom', dy=-5).encode(text='count:Q'),
    bar_payment + bar_payment.mark_text(align='center', baseline='bottom', dy=-5).encode(text='count:Q'),
    bar_triptype + bar_triptype.mark_text(align='center', baseline='bottom', dy=-5).encode(text='count:Q')
).resolve_scale(y='independent')

<h5>Insight</h5>

- Tarif yang sering digunakan penumpang adalah <i>standart rate</i> yang artinya banyak penumpang menggunakan taksi untuk berpergian di dalam kota saja.
- Tipe pembayaran yang paling sering digunakan penumpang adalah credit card.
- Banyak penumpang yang masih memanggil taksi di jalanan.

In [ ]:
agg_distance = df.groupby('distance_bins').agg(count=('distance_bins', 'count'), median=('trip_distance', 'median')).sort_values(by='count', ascending=False).reset_index()
agg_duration = df.groupby('duration_bins').agg(count=('duration_bins', 'count'), median=('trip_duration', 'median')).sort_values(by='count', ascending=False).reset_index()

In [ ]:
# Buat diagram batang untuk proporsi jarak.
bar_distance = alt.Chart(agg_distance).mark_bar().encode(
    x=alt.X('distance_bins:N', title='Distance Bins'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.Color('distance_bins:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])),  # Adjust colors as needed
    tooltip=['distance_bins:N', 'count:Q']
).properties(
    width=920,
    height=500,
    title='Distance Proportion'
)

# Tambahkan label teks ke batang untuk bagan jarak.
text_distance = bar_distance.mark_text(
    align='center',
    baseline='middle',
    dx=0,  # Sesuaikan posisi horizontal teks jika diperlukan.
    dy=-5,  # Sesuaikan posisi vertikal teks jika diperlukan.
).encode(
    text='count:Q'
)

# Gabungkan diagram batang dan label teks untuk jarak.
chart_distance = (bar_distance + text_distance).properties(title='Distance Proportion')

# Buat diagram batang untuk proporsi durasi.
bar_duration = alt.Chart(agg_duration).mark_bar().encode(
    x=alt.X('duration_bins:N', title='Duration Bins'),
    y=alt.Y('count:Q', title='Count'),
    color=alt.Color('duration_bins:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])),  # Adjust colors as needed
    tooltip=['duration_bins:N', 'count:Q']
).properties(
    width=920,
    height=500,
    title='Duration Proportion'
)

# Tambahkan label teks ke batang untuk diagram durasi.
text_duration = bar_duration.mark_text(
    align='center',
    baseline='middle',
    dx=0,  # Sesuaikan posisi horizontal teks jika diperlukan.
    dy=-5,  # Sesuaikan posisi vertikal teks jika diperlukan.
).encode(
    text='count:Q'
)

# Gabungkan diagram batang dan label teks untuk durasi.
chart_duration = (bar_duration + text_duration).properties(title='Duration Proportion')

# Tampilkan diagram batang dengan label.
(chart_distance | chart_duration)

<h5>Insight</h5>

- Kebanyakan penumpang menggunakan taksi untuk menempuh perjalanan sejauh < 2 miles.
- Kebanyakan penumpang lama dalam perjalan taxi sekitar 11-20 menit.

<h5></5>

In [ ]:
# Ekstrak hari dari waktu_penjemputan.
df['pickup_day'] = df['lpep_pickup_datetime'].dt.day

# Kelompokkan berdasarkan hari dan hitung jumlah total per hari.
daily_total = df.groupby('pickup_day')['total_amount'].sum().reset_index()

# Hitung jumlah total_jumlah untuk semua hari.
total_amount_sum = daily_total['total_amount'].sum()

# Bagan garis untuk jumlah total per hari.
line_chart = alt.Chart(daily_total).mark_line().encode(
    x='pickup_day:O',
    y='total_amount:Q',
    tooltip=['pickup_day:O', 'total_amount:Q', alt.Tooltip('total_amount_sum:Q', title='Total Amount Sum')]
).transform_calculate(
    total_amount_sum=str(total_amount_sum)  # Konversikan ke string untuk ditampilkan di tooltip.
).properties(
    width=2000,
    height=500,
    title='Total Amount per Day'
)

line_chart

<h5>Insight</h5>

- Tanggal dengan jumlah penghasilan terbanyak ada di tanggal 19 sebanyak $44.177.
- Total Pendapatan bulan januari sebanyak $1.104.576.

<h1>Conclusion and Recommendation</h1>

<h3>Conclusion</h3>

<b>Permintaan dan preferensi pelanggan dan total pendapatan.</b>

- Terdapat pola permintaan pada bulan Januari, Permintaan cenderung naik selama weekdays kemudian menurun ketika memasuki hari weekend.
- Permintaan taksi terendah pada 1 Januari 2023 dan permintaan taksi paling tinggi di bulan januari terjadi pada hari Rabu, 25 Januari 2023.
- Permintan taksi terendah terjadi pada jam 00:00 - 06:00, dan permintaan taksi tertinggi terjadi pada jam 15:00 - 18:00.
- Id 74 adalah tempat dengan jumlah pengguna taksi tertinggi.
- Pelanggan lebih cenderung menggunakan taksi di dalam kota saja dengan tarif standar, masih sering memanggil taksi di pinggir jalan dan kebanyakan pelanggan menggunakan tipe pembayaran credit card.
- Durasi perjalanan yang paling umum adalah antara 11-20 menit dan jarak yang kurang dari 2 mil.
- Total Pendapatan bulan januari sebanyak $1.104.576.

<h3>Recommendation</h3>

<b>Penjadwalan armada taksi dan kerjasama dengan toko/penjual obat-obat an.</b>

Pastikan tersedianya armada taksi yang cukup pada weekdays, terutama antara hari Senin hingga Jumat, dengan fokus lebih pada hari Rabu dan Kamis. kurangi operasi taksi pada hari weekend dan pada waktu tengah malam hingga dini hari. Tambahkan lebih banyak armada di sore hari pada pukul 15:00 hingga 18:00. Dengan asumsi bahwa jam 15:00 hingga 18:00 adalah jam dimana penumpang adalah orang yang pulang dari pekerjaannya, dimana penumpang sedang merasa capek-capeknya, merasa lemas, pusing dan tidak enak badan, perusahaan bisa menawarkan kerjasama dengan perusahaan, toko atau penjual obat-obatan untuk memasarkan produk-produk kesehatan seperti vitamin dengan memasangkan iklan di dalam taksi seperti di dasbor taksi atau mungkin memasangkan iklan di pintu taksi untuk meningkatkan profit perusahaan.


<b>Opsi Pembayaran dan pemanggilan taksi.</b>

menurut saya credit ini berarti cashless, jadi tidak perlu penggunaan kartu credit dari bank saja tapi bisa juga menggunakan e-money atau mungkin perusahaan bisa membuat apps nya sendiri dan membuat sistem e-money nya sendiri seperti go-pay atau bisa juga kredit seperti shopee pay later. beri keuntungan untuk pelanggan yang menggunakan apps dan e-money perusahaan seperti diskon pembayaran taksi atau mungkin bila merekomendasikan ke orang-orang yang dikenal nya maka bisa mendapatkan voucher untuk membeli produk obat-obatan dari perusahaan yang bekerjamasam dengan perusahaan kita agar tertarik menggunakan apps dan e-money perusahaan. di apps kita juga bisa di buat untuk melakukan pemanggilan taksi dan dengan keuntungan yang sebelumnya disebut diharapkan bisa merubah cara orang untuk mencari, memanggil dan menggunakan taksi. 